In [8]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

import json

from torch import cuda
from pyvi.ViTokenizer import tokenize
from langchain.embeddings.huggingface import HuggingFaceEmbeddings

# embed_model_id = 'sentence-transformers/all-MiniLM-L6-v2'
embed_model_id = 'all-MiniLM-L6-v2'

device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'
# device = 'cpu'
cache_dir = "../cache/"

DB_SAVE_NAME = f"endoc_{embed_model_id.split('/')[-1].replace('.','-')}"
DOCUMENT_DIR = "../datasets/KALAPA_ByteBattles_2023_MEDICAL_Set1/processed/"

## Retriever

In [2]:
embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'batch_size': 32, "normalize_embeddings": True, "device": device},
    cache_folder=cache_dir
)

docs = [
    "this is one document",
    "and another document"
]

embeddings = embed_model.embed_documents(docs)

print(
    f"We have {len(embeddings)} doc embeddings, each with "
    f"a dimensionality of {len(embeddings[0])}."
)

We have 2 doc embeddings, each with a dimensionality of 384.


In [3]:
from langchain.schema import Document
from langchain.vectorstores import FAISS
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

faiss_docs = []
for filename in sorted(os.listdir(DOCUMENT_DIR)):
    filepath = os.path.join(DOCUMENT_DIR, filename)
    with open(filepath, "r", encoding="utf-8") as f:
        file_data = json.load(f)
        
        faiss_docs.append(Document(
            page_content=file_data["content"],
            metadata={"filename": filename, "path": filepath, "title": file_data["title"], "category": file_data["category"], "abstract": file_data["abstract"], "subsections": file_data["subsections"]}
        ))

embed_model = HuggingFaceEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'batch_size': 32, "normalize_embeddings": True, "device": device },
    cache_folder=cache_dir
)

db = FAISS.from_documents(
    documents=faiss_docs,
    embedding=embed_model,
)

In [9]:
db.save_local(DB_SAVE_NAME)

In [5]:
from langchain.vectorstores import FAISS
from langchain.embeddings.sentence_transformer import SentenceTransformerEmbeddings

device = 'cpu'
embed_model = SentenceTransformerEmbeddings(
    model_name=embed_model_id,
    model_kwargs={'device': device},
    encode_kwargs={'batch_size': 32, "normalize_embeddings": True, "device": device },
    cache_folder=cache_dir
)

loaded_db = FAISS.load_local(
    DB_SAVE_NAME,
    embeddings=embed_model
)

In [6]:
query = """
Đâu là triệu chứng của bệnh van tim?
A. Khó thở
B. Tăng cân nhanh chóng
C. Vàng da
D. Rụng tóc
"""

result = loaded_db.similarity_search(query=query, k=1)
result

[Document(page_content='Bệnh mạch vành: Nguyên nhân, dấu hiệu và cách điều trị\n\nBệnh mạch vành là dạng bệnh tim mạch phổ biến nhất ở người lớn, là một trong những nguyên nhân gây tử vong hàng đầu trên thế giới. Tại Việt Nam, tỷ lệ bệnh nhân tử vong do suy mạch vành khá cao, chiếm từ 11% – 36% và đang có dấu hiệu tăng dần mỗi năm. Bài viết được tư vấn chuyên môn bởi PGS.TS.BS Phạm Nguyễn Vinh, Giám đốc Trung tâm Tim mạch, Bệnh viện Đa khoa Tâm Anh TP.HCM\nĐau thắt ngực, khó thở là biểu hiện dễ nhận thấy nhất của bệnh mạch vành\n\nBệnh mạch vành là gì?\nBệnh mạch vành hay suy mạch vành là tình trạng các mạch máu chính cung cấp máu cho tim bị hẹp hoặc tắc nghẽn do mảng xơ vữa, khiến lưu lượng máu đến tim giảm đi. (1)\nBệnh có 2 thể là hội chứng động mạch vành cấp và bệnh động mạch vành mạn:\nHội chứng động mạch vành cấp là tình trạng bao gồm cơn đau thắt ngực không ổn định và nhồi máu cơ tim cấp, xảy ra do mạch vành bị tắc nghẽn hoàn toàn, cấp tính do mảng xơ vữa bị nứt vỡ hoặc do huyết

In [7]:
context = result[0].page_content
print(context)

Bệnh mạch vành: Nguyên nhân, dấu hiệu và cách điều trị

Bệnh mạch vành là dạng bệnh tim mạch phổ biến nhất ở người lớn, là một trong những nguyên nhân gây tử vong hàng đầu trên thế giới. Tại Việt Nam, tỷ lệ bệnh nhân tử vong do suy mạch vành khá cao, chiếm từ 11% – 36% và đang có dấu hiệu tăng dần mỗi năm. Bài viết được tư vấn chuyên môn bởi PGS.TS.BS Phạm Nguyễn Vinh, Giám đốc Trung tâm Tim mạch, Bệnh viện Đa khoa Tâm Anh TP.HCM
Đau thắt ngực, khó thở là biểu hiện dễ nhận thấy nhất của bệnh mạch vành

Bệnh mạch vành là gì?
Bệnh mạch vành hay suy mạch vành là tình trạng các mạch máu chính cung cấp máu cho tim bị hẹp hoặc tắc nghẽn do mảng xơ vữa, khiến lưu lượng máu đến tim giảm đi. (1)
Bệnh có 2 thể là hội chứng động mạch vành cấp và bệnh động mạch vành mạn:
Hội chứng động mạch vành cấp là tình trạng bao gồm cơn đau thắt ngực không ổn định và nhồi máu cơ tim cấp, xảy ra do mạch vành bị tắc nghẽn hoàn toàn, cấp tính do mảng xơ vữa bị nứt vỡ hoặc do huyết khối.; Ngược lại, bệnh mạch vàn